# Download Data for baseline

In [13]:
import wget
url = r"https://tickettagger.blob.core.windows.net/datasets/dataset-labels-top3-30k-real.txt"
output = r"../dataset/dataset-labels-top3-30k-real.txt"
wget.download(url,output)

url = r"https://gist.githubusercontent.com/rafaelkallis/6aa281b00d73d77fc843bd34f8184854/raw/8c10ebf2fd6f937f8667c660ea33d122bac739eb/issues.txt"
output = r"../dataset/dataset_balanced.txt"
wget.download(url,output)

'../dataset/dataset-labels-top3-30k-real (1).txt'

## look at pivot table of data from other repos

In [90]:
import pandas as pd
# custom data set
data_path = r"..\dataset\multiple_projects_benchmark_data.csv"
data_out_path = r"..\dataset\CLEAN_multiple_projects_benchmark_data.csv"
df = pd.read_csv(data_path)
df = df.dropna()
df.head()


,f0_,repo_name
0,__label__bug Single most recently added Custom...,https://api.github.com/repos/qgis/QGIS
1,__label__bug layer style effects not compatibl...,https://api.github.com/repos/qgis/QGIS
2,__label__bug QGIS 3.10.0 fails to launch on Wi...,https://api.github.com/repos/qgis/QGIS
3,__label__bug TIN interpolation in Model Builde...,https://api.github.com/repos/qgis/QGIS
4,__label__bug Map Tip Position / Follow the Hyp...,https://api.github.com/repos/qgis/QGIS


In [91]:
# import pandas as pd

# data_path = r"..\dataset\dataset_balanced.txt"
# data_out_path = r"..\dataset\CLEAN_balanced_data.csv"

# df = pd.read_csv(data_path,names=["f0_"])
# df = df.dropna()
# df = df.drop(index=0).reset_index()
# df.head()

In [92]:
import pandas as pd

# data_path = r"..\dataset\dataset-labels-top3-30k-real.txt"
# data_out_path = r"..\dataset\CLEAN_UNbalanced_data.csv"

# df = pd.read_csv(data_path,names=["f0_"])
# df = df.dropna()
# df = df.drop(index=0).reset_index()
# df.head()

In [93]:
df['labels'] = df['f0_'].str.split(" ").map(lambda x: x[0])
df = df.rename(columns={'f0_':'descriptions'})
print(df.columns)
df.head()

Index(['descriptions', 'repo_name', 'labels'], dtype='object')


,descriptions,repo_name,labels
0,__label__bug Single most recently added Custom...,https://api.github.com/repos/qgis/QGIS,__label__bug
1,__label__bug layer style effects not compatibl...,https://api.github.com/repos/qgis/QGIS,__label__bug
2,__label__bug QGIS 3.10.0 fails to launch on Wi...,https://api.github.com/repos/qgis/QGIS,__label__bug
3,__label__bug TIN interpolation in Model Builde...,https://api.github.com/repos/qgis/QGIS,__label__bug
4,__label__bug Map Tip Position / Follow the Hyp...,https://api.github.com/repos/qgis/QGIS,__label__bug


In [94]:
df.groupby(['repo_name','labels']).agg({'descriptions':'count'})
# df.groupby(['labels']).agg({'descriptions':'count'})

descriptions
repo_name                                          labels                            
https://api.github.com/repos/Kaiserreich/Kaiser... __label__bug                  7476
https://api.github.com/repos/Microsoft/vscode      __label__bug                  8005
                                                   __label__question              170
https://api.github.com/repos/Regalis11/Barotrauma  __label__bug                  3774
                                                   __label__question                7
https://api.github.com/repos/dbeaver/dbeaver       __label__bug                  4246
                                                   __label__enhancement           467
                                                   __label__question             1395
https://api.github.com/repos/godotengine/godot     __label__bug                 11591
                                                   __label__enhancement          1945
https://api.github.com/repos/hashicorp/terraform   __label__bug                  5008
                                                   __label__enhancement          1531
                                                   __label__question              277
https://api.github.com/repos/libsdl-org/SDL        __label__bug                  3183
                                                   __label__enhancement           196
https://api.github.com/repos/microsoft/vscode      __label__bug                  8291
https://api.github.com/repos/qgis/QGIS             __label__bug                  4612
https://api.github.com/repos/rstudio/rstudio       __label__bug                  1997
                                                   __label__enhancement           757
https://api.github.com/repos/symfony/symfony       __label__bug                  4045
                                                   __label__enhancement           120
https://api.github.com/repos/tgstation/tgstation   __label__bug                  5575

In [95]:
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import re

In [96]:
# remove any non-english samples
# reference: https://stackoverflow.com/a/150078/9960491
reg_obj = re.compile(r'[^\u0000-\u007F]+', re.UNICODE)
def is_english(desc):
    return (False if reg_obj.match(desc) else True)

df['is_english'] = df['descriptions'].apply(is_english)

In [97]:
# remove the stopwords, emojis from the text and convert it into lower case
def neatify_text(text):
    text = str(text).lower()
    text = nfx.remove_stopwords(text)
    text = nfx.remove_emojis(text)
#     text = nfx.remove_puncts(text)
    return text

df['descriptions'] = df['descriptions'].apply(neatify_text)

In [98]:
# split dataset to train and dev samples
RANDOM_SEED = 12
x_train, x_test, y_train, y_test = train_test_split(df['descriptions'], df['labels'],
                                                 test_size=0.3,
                                                 random_state=RANDOM_SEED)
df_train = pd.concat([x_train, y_train], axis=1).reset_index(drop=True)
df_eval = pd.concat([x_test, y_test], axis=1).reset_index(drop=True)


In [99]:
df.groupby(['labels']).agg({'descriptions':'count'})

,descriptions
labels,
__label__bug,67803
__label__enhancement,5016
__label__question,1849


In [100]:
# # save train and eval dataframes to csv
df_train.to_csv(r"../dataset/Mult_train.csv")
df_eval.to_csv(r"../dataset/Mult_eval.csv")
df.to_csv(data_out_path)

In [89]:
# save train and eval dataframes to csv
# df_train.to_csv(r"../dataset/balanced_train.csv")
# df_eval.to_csv(r"../dataset/balanced_eval.csv")
# df.to_csv(data_out_path)

In [ ]:
# save train and eval dataframes to csv
# df_train.to_csv(r"../dataset/unbalanced_train.csv")
# df_eval.to_csv(r"../dataset/unbalanced_eval.csv")
# df.to_csv(data_out_path)